In [ ]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [ ]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [ ]:
ls -lh /users/trush/CSC496/DataParallelComputing/data/ml-latest

In [ ]:
!ls

In [ ]:
ratings = sc.textFile("/users/trush/CSC496/DataParallelComputing/data/ml-latest/ratings.csv")

In [ ]:
ratings.take(5)

In [ ]:
ratings.cache()

In [ ]:
%%time
ratings.count()

In [ ]:
%%time
ratings.count()

In [ ]:
%%time
ratings.count()

# Average ratings over the years of every movie

In [ ]:
ratingHeader = ratings.first()
print(ratingHeader)

In [ ]:
ratingsOnly = ratings.filter(lambda x : x != ratingHeader)
ratingsOnly.take(5)

In [ ]:
s = '1,307,3.5,1256677221'
s.split(",")[2]

In [ ]:
movieRatings = ratingsOnly.map(lambda line : (line.split(",")[1], float(line.split(",")[2])))
movieRatings.take(5)

## Possible approaches in aggregating data
 - groupByKey and mapValues
 - reduceByKey and countByKey

_groupByKey_ and _mapValues_

In [ ]:
groupByKeyRatings = movieRatings.groupByKey()
#groupByKeyRatings
groupByKeyRatings.take(5)

In [ ]:
mapValuesToListRatings = groupByKeyRatings.mapValues(list)
#mapValuesToListRatings
mapValuesToListRatings.take(5)

In [ ]:
avgRatings01 = mapValuesToListRatings.mapValues(lambda V: sum(V) / float(len(V)))
#avgRatings01
avgRatings01.take(5)

In [ ]:
test = [2.0, 4.0, 3.0]
sum(test) / len(test)

_reduceByKey_ and _countByKey_

In [ ]:
countsByKey = movieRatings.countByKey()
countsByKey

In [ ]:
def sumValues(x, y):
    return (x + y)

In [ ]:
sumRatings = movieRatings.reduceByKey(sumValues)
sumRatings.take(10)

In [ ]:
avgRatings02 = sumRatings.map(lambda x: (x[0], x[1]/countsByKey.get(x[0])))
avgRatings02.take(10)

In [ ]:
movies = sc.textFile("/users/trush/CSC496/DataParallelComputing/data/ml-latest/movies.csv")
movies.take(5)

In [ ]:
movieHeader = movies.first()
movieInfo = movies.filter(lambda x: x != movieHeader)
movieInfo.take(5)

In [ ]:
# rsplit handles movies with commas in the title
movieTitles = movieInfo.map(lambda line: (line.split(",",1)[0], line.rsplit(",",1)[0].split(",",1)[1]))
movieTitles.take(5)

In [ ]:
s = '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy'
print(s.split(","))
print(s.rsplit(",",1)[0])
print(s.rsplit(",",1)[0].split(",",1)[1])

In [ ]:
augmentedRatings = avgRatings01.join(movieTitles)
augmentedRatings.take(5)

In [ ]:
augmentedRatings.takeOrdered(50, key = lambda x: -x[1][0])

## Challenge:
   report the average ratings for movies with ratings higher than 3.75 and the number of ratings total more than 100

In [ ]:
# Number of Ratings per title
# countsByKey

# Average Rating per title
# avgRatings02

avgRatings03 = sumRatings.map(lambda x: (x[0], x[1]/countsByKey.get(x[0]), countsByKey.get(x[0])))
avgRatings03 = avgRatings03.filter(lambda x: x[1] > 3.75)
avgRatings03 = avgRatings03.filter(lambda x: x[2] > 100)
avgRatings03.take(5)